<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz

--2020-02-28 22:44:17--  http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz
Resolving qwone.com (qwone.com)... 108.20.201.166
Connecting to qwone.com (qwone.com)|108.20.201.166|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17332201 (17M) [application/x-gzip]
Saving to: ‘20news-19997.tar.gz’

20news-19997.tar.gz 100%[===================>]  16.53M  6.88MB/s    in 2.4s    

2020-02-28 22:44:19 (6.88 MB/s) - ‘20news-19997.tar.gz’ saved [17332201/17332201]



In [0]:
!gunzip 20news-19997.tar.gz

In [3]:
!ls

20news-19997.tar  sample_data


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
count_vect = CountVectorizer()
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data) #Learn the vocabulary dictionary and return term-document matrix.
X_train_counts.shape
X_train_counts

<2257x35788 sparse matrix of type '<class 'numpy.int64'>'
	with 365886 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.metrics import *
twenty_test = fetch_20newsgroups(subset='test', categories=categories, 
                                 shuffle=True, random_state=0)
X_test = count_vect.transform(twenty_test.data) # Extract token counts out of raw text documents using the vocabulary fitted with fit 
print(X_test.shape)


(1502, 35788)


In [74]:
features = pd.DataFrame(X_train_counts)
answers = pd.DataFrame(twenty_train.target)
print(len(features))
len(answers)

2257


2257

In [0]:
def scores(model):

  predicted = model.predict(X_test)
  acc = accuracy_score(predicted, twenty_test.target)
  micro_f1 = f1_score(predicted, twenty_test.target, average = 'micro')
  micro_p = precision_score(predicted, twenty_test.target, average = 'micro')
  micro_r = recall_score(predicted, twenty_test.target, average = 'micro')
  macro_f1 = f1_score(predicted, twenty_test.target, average = 'macro')
  macro_p = precision_score(predicted, twenty_test.target, average = 'macro')
  macro_r = recall_score(predicted, twenty_test.target, average = 'macro')
  print('acc={0:1.4f}'.format(acc))
  print('micro F1={0:1.4f}, micro P={1:1.4f}, micro R={2:1.4f}'.format(micro_f1, micro_p, micro_r))
  print('macro F1={0:1.4f}, macro P={1:1.4f}, macro R={2:1.4f}\n'.format(macro_f1, macro_p, macro_r))

In [0]:
from sklearn.model_selection import train_test_split, KFold

In [0]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=0)
stratified_folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
repeated_folds = RepeatedStratifiedKFold(n_splits=n_fold, n_repeats=20, random_state=0)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
kneighbors = KNeighborsClassifier().fit(X_train_counts, twenty_train.target)

In [12]:
parameter_grid = {'weights' : ['uniform', 'distance'],
                  'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
                  'leaf_size' : [20, 15, 25],
                  'n_neighbors': [5,4,6]
                 }

grid_search = GridSearchCV(kneighbors, param_grid=parameter_grid, cv=folds)
grid_search.fit(X_train_counts, twenty_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklear

Best score: 0.7275339233038347
Best parameters: {'algorithm': 'auto', 'leaf_size': 20, 'n_neighbors': 4, 'weights': 'distance'}


In [0]:
chosen_kneighbors = KNeighborsClassifier(algorithm='auto', leaf_size=20, n_neighbors=4, weights='distance').fit(X_train_counts, twenty_train.target)

In [14]:
scores(chosen_kneighbors)

acc=0.5732
micro F1=0.5732, micro P=0.5732, micro R=0.5732
macro F1=0.5705, macro P=0.5712, macro R=0.6386



In [0]:
from sklearn import linear_model

In [16]:
lr = linear_model.LogisticRegression().fit(X_train_counts, twenty_train.target)

parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'solver' : ['liblinear', 'saga'],
                  'C' : [0.001, 0.01, 0.08],
                  'max_iter': [2,10]
                 }

grid_search = GridSearchCV(lr, param_grid=parameter_grid, cv=folds)
grid_search.fit(X_train_counts, twenty_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sa

Best score: 0.9570285152409046
Best parameters: {'C': 0.08, 'class_weight': 'balanced', 'max_iter': 10, 'penalty': 'l2', 'solver': 'liblinear'}


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [17]:
chosen_lr = linear_model.LogisticRegression(C=0.08, class_weight='balanced', max_iter=10, penalty='l2', solver='liblinear').fit(X_train_counts, twenty_train.target)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [18]:
scores(chosen_lr)

acc=0.8955
micro F1=0.8955, micro P=0.8955, micro R=0.8955
macro F1=0.8927, macro P=0.8916, macro R=0.8995



In [0]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X_train_counts, twenty_train.target)


In [20]:


parameter_grid = {'criterion' : ['gini', 'entropy'],
                  'splitter' : ['best', 'random'],
                  'min_samples_split' : [1, 2, 3],
                  'max_features' : ['auto', 'sqrt', 'log2'],
                  'ccp_alpha': [1.0, 2.0, 0.0]
                 }

grid_search = GridSearchCV(tree, param_grid=parameter_grid, cv=folds)
grid_search.fit(X_train_counts, twenty_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integ

Best score: 0.6818780727630285
Best parameters: {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_split': 2, 'splitter': 'best'}


In [0]:
chosen_tree = DecisionTreeClassifier(ccp_alpha=0.0, criterion='gini', max_features='sqrt', min_samples_split=3,\
                                     splitter='best').fit(X_train_counts, twenty_train.target)

In [22]:
scores(chosen_tree)

acc=0.5666
micro F1=0.5666, micro P=0.5666, micro R=0.5666
macro F1=0.5603, macro P=0.5615, macro R=0.5635



In [23]:
!pip install eli5

     |████████████████████████████████| 112kB 2.6MB/s 


In [24]:
!pip install shap

     |████████████████████████████████| 276kB 2.9MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp36-cp36m-linux_x86_64.whl size=394156 sha256=8de3f1e148ef65bdb75ac817e3e105520ed2b5fc4d84b61f68a346d7380e537a
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap


In [25]:
import eli5
from eli5.sklearn import PermutationImportance
import shap

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [82]:
eli5.formatters.as_dataframe.explain_weights_df(chosen_tree)

,feature,weight
0,x4992,0.027170
1,x26169,0.015896
2,x8553,0.015272
3,x32135,0.014565
4,x5340,0.013878
...,...,...
35783,x23730,0.000000
35784,x23731,0.000000
35785,x23732,0.000000
35786,x23734,0.000000


In [0]:
top_features = [int(i[1:]) for i in eli5.formatters.as_dataframe.explain_weights_df(model).feature if  i]

In [81]:
perm = PermutationImportance(chosen_tree, random_state=1).fit( X_train_counts.toarray, twenty_train.target)

ValueError: ignored

In [64]:
chosen_tree.feature_importances_[1]
# not_null = index_to_word = {v:k for k,v in chosen_tree.feature_importances_.items()}

0.0

In [0]:
# for i in chosen_tree.feature_importances_:
#   if i:
#     print(i)

In [40]:
eli5.formatters.as_dataframe.explain_weights_df(chosen_lr)

,target,feature,weight
0,0,x19218,0.475887
1,0,x5714,0.421892
2,0,x21167,0.345379
3,0,x5718,0.343910
4,0,x23677,0.324737
...,...,...,...
143151,3,x5529,-0.301885
143152,3,x15699,-0.372903
143153,3,x25663,-0.516741
143154,3,x23122,-0.596738


In [0]:
top_features = [int(i[1:]) for i in eli5.formatters.as_dataframe.explain_weights_df(chosen_lr).feature if 'BIAS' not in i]

In [36]:
top_features

[4992,
 26169,
 8553,
 32135,
 5340,
 25234,
 32139,
 20361,
 28994,
 30117,
 24784,
 5529,
 7271,
 2148,
 19218,
 18501,
 16881,
 28497,
 28473,
 8090,
 14247,
 4039,
 15959,
 8945,
 22337,
 25483,
 1102,
 18774,
 28488,
 12443,
 8105,
 8559,
 7435,
 33256,
 12661,
 28274,
 9072,
 14281,
 8017,
 32142,
 15173,
 13672,
 17366,
 19251,
 34289,
 8544,
 139,
 16607,
 11505,
 34230,
 32493,
 8557,
 10604,
 14819,
 29224,
 35006,
 14772,
 23122,
 12153,
 6298,
 33302,
 24267,
 35350,
 10787,
 6597,
 11493,
 33597,
 23741,
 26636,
 4237,
 8718,
 8362,
 18542,
 25154,
 22200,
 23102,
 19298,
 24024,
 23052,
 5730,
 896,
 15319,
 22652,
 6427,
 15309,
 9183,
 28736,
 35638,
 9378,
 11729,
 21254,
 23870,
 12626,
 8035,
 24296,
 7921,
 6599,
 34252,
 4471,
 20277,
 30208,
 21855,
 23478,
 24895,
 3958,
 25612,
 10558,
 19951,
 15232,
 5637,
 35057,
 32643,
 29474,
 12764,
 37,
 14037,
 14079,
 27504,
 15618,
 5717,
 25765,
 27447,
 13521,
 2946,
 26809,
 32438,
 2258,
 19647,
 8609,
 32417,
 32251,
 6355,
 17302,
 22764,
 26355,
 29313,
 15682,
 12283,
 32607,
 19348,
 19604,
 21743,
 7941,
 34306,
 31366,
 4206,
 32152,
 22582,
 5807,
 7800,
 2694,
 33212,
 11520,
 8583,
 24020,
 20979,
 24676,
 21311,
 16701,
 35313,
 32526,
 17666,
 18816,
 20978,
 23610,
 30941,
 29285,
 24481,
 13998,
 9110,
 7517,
 24945,
 5289,
 24052,
 17556,
 5549,
 14670,
 33915,
 26532,
 33847,
 30048,
 4748,
 22018,
 31849,
 34987,
 20253,
 32270,
 8474,
 21339,
 18337,
 15576,
 32179,
 4882,
 5698,
 35299,
 35269,
 10729,
 23250,
 19458,
 10936,
 31272,
 30173,
 19766,
 33550,
 20463,
 26556,
 5783,
 11152,
 4741,
 31737,
 5196,
 16711,
 6309,
 34807,
 7395,
 3806,
 15383,
 4218,
 16972,
 29254,
 35157,
 19670,
 23814,
 24353,
 31785,
 18692,
 16873,
 17219,
 6358,
 11726,
 33249,
 25143,
 31610,
 33773,
 31975,
 35354,
 21415,
 10620,
 16674,
 32131,
 18715,
 27928,
 13726,
 29303,
 16254,
 24595,
 8159,
 19467,
 17945,
 11840,
 13021,
 336,
 5128,
 28693,
 13168,
 57,
 10790,
 22074,
 7770,
 10977,
 32083,
 33940,
 11900,
 33450,
 24981,
 27734,
 20289,
 4690,
 22564,
 35146,
 23292,
 14018,
 23872,
 28045,
 25889,
 27575,
 20198,
 30235,
 32257,
 5010,
 18793,
 34472,
 28089,
 32775,
 7325,
 12597,
 12894,
 24299,
 1701,
 7818,
 1057,
 23063,
 18100,
 25406,
 27053,
 6744,
 10915,
 22487,
 23280,
 21801,
 7480,
 28876,
 15776,
 13717,
 32202,
 34299,
 23677,
 5788,
 15599,
 21528,
 13552,
 18067,
 14612,
 20116,
 30862,
 29601,
 22743,
 4936,
 12366,
 9743,
 13512,
 30359,
 18474,
 1119,
 3157,
 15386,
 6835,
 12558,
 29516,
 22107,
 652,
 6437,
 34836,
 27195,
 30703,
 19969,
 6570,
 16537,
 22475,
 16335,
 3766,
 34593,
 29350,
 13503,
 9961,
 2585,
 1471,
 30808,
 17268,
 33891,
 35293,
 13379,
 5583,
 7747,
 1318,
 15042,
 20847,
 11312,
 8009,
 31434,
 9046,
 12439,
 7109,
 15600,
 27238,
 17050,
 21778,
 21386,
 31723,
 28573,
 33872,
 9932,
 9347,
 12988,
 25642,
 8795,
 21572,
 23173,
 34826,
 12036,
 4722,
 30696,
 8264,
 35218,
 5261,
 32042,
 25262,
 24970,
 35132,
 16102,
 28871,
 34727,
 22613,
 29128,
 23260,
 19681,
 34120,
 30469,
 31528,
 13456,
 1341,
 24157,
 16744,
 28136,
 31637,
 18580,
 26097,
 16211,
 23915,
 25656,
 31742,
 2333,
 23632,
 14948,
 14869,
 30330,
 27075,
 30632,
 19899,
 4939,
 35587,
 30277,
 28271,
 34610,
 431,
 3389,
 8064,
 29933,
 26344,
 26398,
 9118,
 5578,
 9925,
 32346,
 7216,
 7071,
 30358,
 32120,
 10752,
 17577,
 23420,
 14980,
 23152,
 17540,
 31749,
 33753,
 314,
 27846,
 9718,
 18447,
 35639,
 19737,
 15637,
 35648,
 20573,
 1986,
 7104,
 23112,
 6540,
 33082,
 7331,
 32751,
 18572,
 18911,
 12783,
 35366,
 8707,
 34923,
 4258,
 12354,
 242,
 25264,
 15762,
 18539,
 2814,
 14389,
 28416,
 14814,
 9342,
 29348,
 1120,
 17748,
 33801,
 26986,
 16666,
 6454,
 35312,
 4330,
 12707,
 31935,
 18268,
 23733,
 23940,
 34303,
 31061,
 16273,
 34210,
 9191,
 14877,
 10812,
 27951,
 25670,
 26779,
 25506,
 29336,
 838,
 32847,
 21732,
 32098,
 2426,
 14378,
 21856,
 14801,
 27876,
 8389,
 12096,
 17305,
 1576